In [1]:
import duckdb
import pandas as pd

# Load the CSV file using DuckDB
con = duckdb.connect(database=':memory:')
con.execute("CREATE TABLE ecommerce AS SELECT * FROM '2019-Oct_ecommerce_bigData.csv'")

# Fetch the top 20 rows
duckdb_result = con.execute("SELECT * FROM ecommerce LIMIT 20").fetchdf()

# Display the top 20 rows
print(duckdb_result)

# Display the column types
duckdb_column_types = con.execute("DESCRIBE ecommerce").fetchdf()
print(duckdb_column_types[['column_name', 'column_type']])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

            event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00       view    44600062  2103807459595387724   
1  2019-10-01 00:00:00       view     3900821  2053013552326770905   
2  2019-10-01 00:00:01       view    17200506  2053013559792632471   
3  2019-10-01 00:00:01       view     1307067  2053013558920217191   
4  2019-10-01 00:00:04       view     1004237  2053013555631882655   
5  2019-10-01 00:00:05       view     1480613  2053013561092866779   
6  2019-10-01 00:00:08       view    17300353  2053013553853497655   
7  2019-10-01 00:00:08       view    31500053  2053013558031024687   
8  2019-10-01 00:00:10       view    28719074  2053013565480109009   
9  2019-10-01 00:00:11       view     1004545  2053013555631882655   
10 2019-10-01 00:00:11       view     2900536  2053013554776244595   
11 2019-10-01 00:00:11       view     1005011  2053013555631882655   
12 2019-10-01 00:00:13       view     3900746  2053013552326770905   
13 2019-10-01 00:00:

In [1]:
import duckdb
import time

# Connect to DuckDB
con = duckdb.connect(database=':memory:')

# Load the CSV file
con.execute("CREATE TABLE ecommerce AS SELECT * FROM '2019-Oct_ecommerce_bigData.csv'")

# Function to run a query and print its execution time
def run_query(query, description):
    print(f"\n{description}")
    start_time = time.time()
    result = con.execute(query).fetchall()
    end_time = time.time()
    print(f"Execution time: {end_time - start_time:.4f} seconds")
    print("Result:", result[:10])  # Print only first 10 results to keep output manageable

# List of queries with descriptions
queries = [
    ("SELECT COUNT(*) FROM ecommerce", "1. Total number of events:"),
    ("SELECT COUNT(DISTINCT user_id) FROM ecommerce", "2. Number of unique users:"),
    ("SELECT product_id, COUNT(*) as view_count FROM ecommerce WHERE event_type = 'view' GROUP BY product_id ORDER BY view_count DESC LIMIT 10", "3. Top 10 most viewed products:"),
    ("SELECT SUM(price) as total_revenue FROM ecommerce WHERE event_type = 'purchase'", "4. Total revenue:"),
    ("SELECT AVG(price) as avg_price FROM ecommerce WHERE event_type = 'purchase'", "5. Average purchase price:"),
    ("SELECT event_type, COUNT(*) as event_count FROM ecommerce GROUP BY event_type", "6. Event count by type:"),
    ("SELECT brand, COUNT(*) as view_count FROM ecommerce WHERE event_type = 'view' AND brand IS NOT NULL GROUP BY brand ORDER BY view_count DESC LIMIT 5", "7. Top 5 brands by views:"),
    ("SELECT EXTRACT(HOUR FROM event_time) as hour, COUNT(*) as event_count FROM ecommerce GROUP BY hour ORDER BY hour", "8. Hourly event distribution:"),
    ("WITH views AS (SELECT COUNT(*) as view_count FROM ecommerce WHERE event_type = 'view'), purchases AS (SELECT COUNT(*) as purchase_count FROM ecommerce WHERE event_type = 'purchase') SELECT CAST(purchase_count AS FLOAT) / view_count as conversion_rate FROM views, purchases", "9. Conversion rate:"),
    ("SELECT category_code, SUM(price) as revenue FROM ecommerce WHERE event_type = 'purchase' AND category_code IS NOT NULL GROUP BY category_code ORDER BY revenue DESC LIMIT 10", "10. Top 10 categories by revenue:"),
    ("SELECT user_id, COUNT(DISTINCT user_session) as session_count FROM ecommerce GROUP BY user_id ORDER BY session_count DESC LIMIT 10", "11. Top 10 users by session count:"),
    ("WITH session_times AS (SELECT user_session, MIN(event_time) as start_time, MAX(event_time) as end_time FROM ecommerce GROUP BY user_session) SELECT AVG(EXTRACT(EPOCH FROM (end_time - start_time))) as avg_duration_seconds FROM session_times", "12. Average session duration:"),
    ("WITH product_events AS (SELECT product_id, SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) as views, SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchases FROM ecommerce GROUP BY product_id) SELECT product_id, CAST(purchases AS FLOAT) / views as conversion_ratio FROM product_events WHERE views > 0 ORDER BY conversion_ratio DESC LIMIT 10", "13. Top 10 products by view-to-purchase ratio:"),
    # ("SELECT SUBSTR(event_time, 1, 10) as date, SUM(price) as daily_revenue FROM ecommerce WHERE event_type = 'purchase' GROUP BY date ORDER BY date", "14. Daily revenue trend:"),
    ("WITH journey AS (SELECT user_id, event_type, LAG(event_type, 1) OVER (PARTITION BY user_id ORDER BY event_time) as prev_event, LAG(event_type, 2) OVER (PARTITION BY user_id ORDER BY event_time) as prev_prev_event FROM ecommerce) SELECT prev_prev_event, prev_event, event_type, COUNT(*) as journey_count FROM journey WHERE prev_prev_event IS NOT NULL GROUP BY prev_prev_event, prev_event, event_type ORDER BY journey_count DESC LIMIT 5", "15. Top 5 user journeys:"),
    ("SELECT CASE WHEN price < 50 THEN '0-50' WHEN price < 100 THEN '50-100' WHEN price < 500 THEN '100-500' ELSE '500+' END as price_range, COUNT(*) as purchase_count FROM ecommerce WHERE event_type = 'purchase' GROUP BY price_range ORDER BY price_range", "16. Price range distribution of purchases:"),
    ("WITH user_events AS (SELECT user_id, SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as cart_adds, SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchases FROM ecommerce GROUP BY user_id) SELECT COUNT(*) as users_with_abandoned_carts FROM user_events WHERE cart_adds > purchases", "17. Users with abandoned carts:"),
    ("WITH product_stats AS (SELECT product_id, AVG(price) as avg_price, COUNT(*) as view_count FROM ecommerce WHERE event_type = 'view' GROUP BY product_id) SELECT CORR(avg_price, view_count) as price_view_correlation FROM product_stats", "18. Price-view correlation:"),
    ("SELECT user_id, SUM(price) as total_spend FROM ecommerce WHERE event_type = 'purchase' GROUP BY user_id ORDER BY total_spend DESC LIMIT 10", "19. Top 10 most valuable customers:"),
    ("WITH user_categories AS (SELECT DISTINCT user_id, category_code FROM ecommerce WHERE event_type = 'view' AND category_code IS NOT NULL), category_pairs AS (SELECT a.category_code as category1, b.category_code as category2, COUNT(DISTINCT a.user_id) as common_users FROM user_categories a JOIN user_categories b ON a.user_id = b.user_id AND a.category_code < b.category_code GROUP BY a.category_code, b.category_code) SELECT category1, category2, common_users, common_users / CAST((SELECT COUNT(DISTINCT user_id) FROM user_categories WHERE category_code = category1) AS FLOAT) as affinity_score FROM category_pairs ORDER BY affinity_score DESC LIMIT 5", "20. Top 5 category affinities:")
]

# Run all queries
for query, description in queries:
    run_query(query, description)

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


1. Total number of events:
Execution time: 0.0120 seconds
Result: [(42448764,)]

2. Number of unique users:
Execution time: 0.7945 seconds
Result: [(3022290,)]

3. Top 10 most viewed products:
Execution time: 0.8538 seconds
Result: [(1004856, 419287), (1004767, 378777), (1005115, 327715), (1004249, 207422), (1004833, 203018), (1005105, 197930), (1004870, 190435), (1002544, 179249), (4804056, 179092), (5100816, 164608)]

4. Total revenue:
Execution time: 0.1430 seconds
Result: [(229957502.2700042,)]

5. Average purchase price:
Execution time: 0.1500 seconds
Result: [(309.5615694037475,)]

6. Event count by type:
Execution time: 0.3474 seconds
Result: [('purchase', 742849), ('cart', 926516), ('view', 40779399)]

7. Top 5 brands by views:
Execution time: 0.6417 seconds
Result: [('samsung', 4806630), ('apple', 3770597), ('xiaomi', 2922650), ('huawei', 1045572), ('lucente', 644283)]

8. Hourly event distribution:
Execution time: 0.1649 seconds
Result: [(0, 306805), (1, 559027), (2, 1069047

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 3.0058 seconds
Result: [(512475445, 7400), (526731152, 2773), (514649263, 2336), (563459593, 1618), (561163588, 1182), (554631275, 860), (512500203, 785), (513847031, 731), (548931675, 681), (537873067, 673)]

12. Average session duration:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 2.7664 seconds
Result: [(1042.0589841095527,)]

13. Top 10 products by view-to-purchase ratio:
Execution time: 1.1318 seconds
Result: [(50300057, 1.0), (26205483, 1.0), (26006029, 1.0), (26009418, 1.0), (26008004, 1.0), (12720815, 1.0), (26011573, 1.0), (29502825, 1.0), (24100063, 1.0), (14701426, 1.0)]

15. Top 5 user journeys:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 12.3726 seconds
Result: [('view', 'view', 'view', 33663206), ('view', 'view', 'cart', 543399), ('purchase', 'view', 'view', 494393), ('view', 'purchase', 'view', 388418), ('view', 'cart', 'view', 354904)]

16. Price range distribution of purchases:
Execution time: 0.1992 seconds
Result: [('0-50', 117070), ('100-500', 397081), ('50-100', 92306), ('500+', 136392)]

17. Users with abandoned carts:
Execution time: 1.4700 seconds
Result: [(223524,)]

18. Price-view correlation:
Execution time: 0.9958 seconds
Result: [(0.03704910489014175,)]

19. Top 10 most valuable customers:
Execution time: 0.2060 seconds
Result: [(519267944, 265569.52), (513117637, 244499.99999999997), (515384420, 210749.77), (530834332, 187128.93000000002), (512386086, 182470.8), (519250600, 174449.35000000003), (513784794, 143821.03000000003), (538216048, 134344.69), (514320330, 119103.99000000002), (553431815, 118762.23999999996)]

20. Top 5 category affinities:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 8.4496 seconds
Result: [('computers.components.power_supply', 'computers.components.videocards', 1741, 0.554811954498291), ('auto.accessories.anti_freeze', 'electronics.smartphone', 74, 0.5138888955116272), ('apparel.sock', 'kids.carriage', 612, 0.5074626803398132), ('computers.peripherals.camera', 'electronics.smartphone', 905, 0.503337025642395), ('computers.ebooks', 'electronics.smartphone', 3196, 0.49078625440597534)]


## Pandas vs DuckDB

In [2]:
import duckdb
import pandas as pd
import time

# Function to measure execution time
def measure_time(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    return result, end_time - start_time

# DuckDB implementation
def duckdb_operations():
    con = duckdb.connect(database=':memory:')
    
    # Load the CSV file
    con.execute("CREATE TABLE ecommerce AS SELECT * FROM '2019-Oct_ecommerce_bigData.csv'")
    
    results = []
    
    # Query 1: Total number of events
    results.append(measure_time(lambda: con.execute("SELECT COUNT(*) FROM ecommerce").fetchall()))
    
    # Query 2: Number of unique users
    results.append(measure_time(lambda: con.execute("SELECT COUNT(DISTINCT user_id) FROM ecommerce").fetchall()))
    
    # Query 3: Top 10 most viewed products
    results.append(measure_time(lambda: con.execute("SELECT product_id, COUNT(*) as view_count FROM ecommerce WHERE event_type = 'view' GROUP BY product_id ORDER BY view_count DESC LIMIT 10").fetchall()))
    
    # Query 4: Total revenue
    results.append(measure_time(lambda: con.execute("SELECT SUM(price) as total_revenue FROM ecommerce WHERE event_type = 'purchase'").fetchall()))
    
    # Query 5: Average purchase price
    results.append(measure_time(lambda: con.execute("SELECT AVG(price) as avg_price FROM ecommerce WHERE event_type = 'purchase'").fetchall()))
    
    # Query 6: Event count by type
    results.append(measure_time(lambda: con.execute("SELECT event_type, COUNT(*) as event_count FROM ecommerce GROUP BY event_type").fetchall()))
    
    con.close()
    return results

# Pandas implementation
def pandas_operations():
    # Load the CSV file
    df = pd.read_csv('2019-Oct_ecommerce_bigData.csv')
    
    results = []
    
    # Query 1: Total number of events
    results.append(measure_time(lambda: len(df)))
    
    # Query 2: Number of unique users
    results.append(measure_time(lambda: df['user_id'].nunique()))
    
    # Query 3: Top 10 most viewed products
    results.append(measure_time(lambda: df[df['event_type'] == 'view']['product_id'].value_counts().nlargest(10).reset_index().values.tolist()))
    
    # Query 4: Total revenue
    results.append(measure_time(lambda: df[df['event_type'] == 'purchase']['price'].sum()))
    
    # Query 5: Average purchase price
    results.append(measure_time(lambda: df[df['event_type'] == 'purchase']['price'].mean()))
    
    # Query 6: Event count by type
    results.append(measure_time(lambda: df['event_type'].value_counts().reset_index().values.tolist()))
    
    return results

# Run and compare
queries = [
    "1. Total number of events:",
    "2. Number of unique users:",
    "3. Top 10 most viewed products:",
    "4. Total revenue:",
    "5. Average purchase price:",
    "6. Event count by type:"
]

print("DuckDB Performance:")
duckdb_results = duckdb_operations()
for query, result in zip(queries, duckdb_results):
    print(f"{query} {result[0]}, Time: {result[1]:.4f} seconds")

print("\nPandas Performance:")
pandas_results = pandas_operations()
for query, result in zip(queries, pandas_results):
    print(f"{query} {result[0]}, Time: {result[1]:.4f} seconds")

# Overall comparison
duckdb_total_time = sum(result[1] for result in duckdb_results)
pandas_total_time = sum(result[1] for result in pandas_results)

print(f"\nTotal time (DuckDB): {duckdb_total_time:.4f} seconds")
print(f"Total time (Pandas): {pandas_total_time:.4f} seconds")
print(f"DuckDB is {pandas_total_time / duckdb_total_time:.2f}x faster than Pandas for these operations")

DuckDB Performance:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

1. Total number of events: [(42448764,)], Time: 0.0212 seconds
2. Number of unique users: [(3022290,)], Time: 1.2935 seconds
3. Top 10 most viewed products: [(1004856, 419287), (1004767, 378777), (1005115, 327715), (1004249, 207422), (1004833, 203018), (1005105, 197930), (1004870, 190435), (1002544, 179249), (4804056, 179092), (5100816, 164608)], Time: 0.9167 seconds
4. Total revenue: [(229957502.27000415,)], Time: 0.1096 seconds
5. Average purchase price: [(309.56156940374643,)], Time: 0.1166 seconds
6. Event count by type: [('cart', 926516), ('purchase', 742849), ('view', 40779399)], Time: 0.2917 seconds

Pandas Performance:
1. Total number of events: 42448764, Time: 0.0000 seconds
2. Number of unique users: 3022290, Time: 3.2735 seconds
3. Top 10 most viewed products: [[1004856, 419287], [1004767, 378777], [1005115, 327715], [1004249, 207422], [1004833, 203018], [1005105, 197930], [1004870, 190435], [1002544, 179249], [4804056, 179092], [5100816, 164608]], Time: 24.2381 seconds
4. T

In [2]:
import duckdb
import polars as pl
import time

# DuckDB setup
duck_conn = duckdb.connect('ecommerce.db')

# Drop the table if it exists
duck_conn.execute("DROP TABLE IF EXISTS ecommerce")

duck_conn.execute("CREATE TABLE ecommerce AS SELECT * FROM read_csv_auto('2019-Oct_ecommerce_bigData.csv')")

# Polars setup
df = pl.read_csv("2019-Oct_ecommerce_bigData.csv")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
import time
def time_execution(func):
    start = time.time()
    result = func()
    end = time.time()
    print(f"Execution time: {end - start:.4f} seconds")
    return result

In [4]:

# Query 1: Count total number of events
print("Query 1: Count total number of events")

duck_result = time_execution(lambda: duck_conn.execute("SELECT COUNT(*) FROM ecommerce").fetchone()[0])
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: len(df))
print(f"Polars result: {polars_result}")


Query 1: Count total number of events
Execution time: 0.5530 seconds
DuckDB result: 42448764
Execution time: 0.0040 seconds
Polars result: 42448764


In [5]:

# Query 2: Count unique users
print("\nQuery 2: Count unique users")

duck_result = time_execution(lambda: duck_conn.execute("SELECT COUNT(DISTINCT user_id) FROM ecommerce").fetchone()[0])
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df['user_id'].n_unique())
print(f"Polars result: {polars_result}")



Query 2: Count unique users


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 3.3075 seconds
DuckDB result: 3022290
Execution time: 1.9898 seconds
Polars result: 3022290


In [5]:

# Query 3: Top 10 most viewed products
print("\nQuery 3: Top 10 most viewed products")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT product_id, COUNT(*) as view_count
    FROM ecommerce
    WHERE event_type = 'view'
    GROUP BY product_id
    ORDER BY view_count DESC
    LIMIT 10
""").fetchall())
print(f"DuckDB result: {duck_result[:3]}...")  # Showing only first 3 results

polars_result = time_execution(lambda: df.filter(pl.col('event_type') == 'view')
                               .groupby('product_id')
                               .agg(pl.count().alias('view_count'))
                               .sort('view_count', descending=True)
                               .limit(10)
                               .collect())
print(f"Polars result: {polars_result[:3]}...")  # Showing only first 3 results



Query 3: Top 10 most viewed products
Execution time: 1.4306 seconds
DuckDB result: [(1004856, 419287), (1004767, 378777), (1005115, 327715)]...


AttributeError: 'DataFrame' object has no attribute 'groupby'

In [6]:

# Query 4: Total revenue by category
print("\nQuery 4: Total revenue by category")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT category_code, SUM(price) as total_revenue
    FROM ecommerce
    WHERE event_type = 'purchase'
    GROUP BY category_code
    ORDER BY total_revenue DESC
    LIMIT 5
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.filter(pl.col('event_type') == 'purchase')
                               .groupby('category_code')
                               .agg(pl.sum('price').alias('total_revenue'))
                               .sort('total_revenue', descending=True)
                               .limit(5)
                               .collect())
print(f"Polars result: {polars_result}")



Query 4: Total revenue by category
Execution time: 2.0859 seconds
DuckDB result: [('electronics.smartphone', 157049623.36999413), (None, 22924937.130000144), ('computers.notebook', 8979887.25), ('electronics.video.tv', 8423407.860000057), ('electronics.clocks', 4818305.46999998)]


AttributeError: 'DataFrame' object has no attribute 'groupby'

In [6]:

# Query 5: Average price by brand
print("\nQuery 5: Average price by brand")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT brand, AVG(price) as avg_price
    FROM ecommerce
    WHERE brand IS NOT NULL
    GROUP BY brand
    ORDER BY avg_price DESC
    LIMIT 5
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.filter(pl.col('brand').is_not_null())
                               .groupby('brand')
                               .agg(pl.mean('price').alias('avg_price'))
                               .sort('avg_price', descending=True)
                               .limit(5)
                               .collect())
print(f"Polars result: {polars_result}")



Query 5: Average price by brand


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Execution time: 3.1551 seconds
DuckDB result: [('climadiff', 2393.1709790209784), ('pinarello', 2382.3405263157897), ('fondital', 2174.6649100719433), ('rado', 2087.4551580831244), ('jumper', 1853.3299999999967)]


: 

In [5]:
import psutil
print(f"Available memory: {psutil.virtual_memory().available / (1024 * 1024 * 1024):.2f} GB")

Available memory: 10.77 GB


In [1]:

# Query 6: User session analysis
print("\nQuery 6: User session analysis")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT user_session, 
           COUNT(*) as event_count, 
           COUNT(DISTINCT event_type) as unique_event_types,
           MIN(event_time) as session_start,
           MAX(event_time) as session_end
    FROM ecommerce
    GROUP BY user_session
    ORDER BY event_count DESC
    LIMIT 5
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.groupby('user_session')
                               .agg([
                                   pl.count().alias('event_count'),
                                   pl.n_unique('event_type').alias('unique_event_types'),
                                   pl.min('event_time').alias('session_start'),
                                   pl.max('event_time').alias('session_end')
                               ])
                               .sort('event_count', descending=True)
                               .limit(5)
                               .collect())
print(f"Polars result: {polars_result}")



Query 6: User session analysis


NameError: name 'time_execution' is not defined

In [ ]:

# Query 7: Daily event counts
print("\nQuery 7: Daily event counts")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT DATE(event_time) as date, COUNT(*) as event_count
    FROM ecommerce
    GROUP BY DATE(event_time)
    ORDER BY date
    LIMIT 5
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.with_columns(pl.col('event_time').cast(pl.Date).alias('date'))
                               .groupby('date')
                               .agg(pl.count().alias('event_count'))
                               .sort('date')
                               .limit(5)
                               .collect())
print(f"Polars result: {polars_result}")

# Query 8: Product funnel analysis
print("\nQuery 8: Product funnel analysis")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT product_id,
           SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) as view_count,
           SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as cart_count,
           SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchase_count
    FROM ecommerce
    GROUP BY product_id
    ORDER BY view_count DESC
    LIMIT 5
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.groupby('product_id')
                               .agg([
                                   pl.sum(pl.col('event_type') == 'view').alias('view_count'),
                                   pl.sum(pl.col('event_type') == 'cart').alias('cart_count'),
                                   pl.sum(pl.col('event_type') == 'purchase').alias('purchase_count')
                               ])
                               .sort('view_count', descending=True)
                               .limit(5)
                               .collect())
print(f"Polars result: {polars_result}")

# Query 9: Price range distribution
print("\nQuery 9: Price range distribution")

duck_result = time_execution(lambda: duck_conn.execute("""
    SELECT 
        CASE 
            WHEN price < 50 THEN 'Under $50'
            WHEN price >= 50 AND price < 100 THEN '$50 - $99'
            WHEN price >= 100 AND price < 500 THEN '$100 - $499'
            WHEN price >= 500 AND price < 1000 THEN '$500 - $999'
            ELSE '$1000 and above'
        END as price_range,
        COUNT(*) as product_count
    FROM ecommerce
    GROUP BY price_range
    ORDER BY product_count DESC
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.with_columns(
    pl.when(pl.col('price') < 50).then('Under $50')
    .when((pl.col('price') >= 50) & (pl.col('price') < 100)).then('$50 - $99')
    .when((pl.col('price') >= 100) & (pl.col('price') < 500)).then('$100 - $499')
    .when((pl.col('price') >= 500) & (pl.col('price') < 1000)).then('$500 - $999')
    .otherwise('$1000 and above')
    .alias('price_range')
)
.groupby('price_range')
.agg(pl.count().alias('product_count'))
.sort('product_count', descending=True)
.collect())
print(f"Polars result: {polars_result}")

# Query 10: User purchase frequency
print("\nQuery 10: User purchase frequency")

duck_result = time_execution(lambda: duck_conn.execute("""
    WITH user_purchases AS (
        SELECT user_id, COUNT(*) as purchase_count
        FROM ecommerce
        WHERE event_type = 'purchase'
        GROUP BY user_id
    )
    SELECT 
        CASE 
            WHEN purchase_count = 1 THEN 'One-time'
            WHEN purchase_count = 2 THEN 'Two-time'
            WHEN purchase_count >= 3 AND purchase_count <= 5 THEN '3-5 times'
            WHEN purchase_count > 5 THEN 'More than 5 times'
        END as purchase_frequency,
        COUNT(*) as user_count
    FROM user_purchases
    GROUP BY purchase_frequency
    ORDER BY user_count DESC
""").fetchall())
print(f"DuckDB result: {duck_result}")

polars_result = time_execution(lambda: df.filter(pl.col('event_type') == 'purchase')
                               .groupby('user_id')
                               .agg(pl.count().alias('purchase_count'))
                               .with_columns(
                                   pl.when(pl.col('purchase_count') == 1).then('One-time')
                                   .when(pl.col('purchase_count') == 2).then('Two-time')
                                   .when((pl.col('purchase_count') >= 3) & (pl.col('purchase_count') <= 5)).then('3-5 times')
                                   .otherwise('More than 5 times')
                                   .alias('purchase_frequency')
                               )
                               .groupby('purchase_frequency')
                               .agg(pl.count().alias('user_count'))
                               .sort('user_count', descending=True)
                               .collect())
print(f"Polars result: {polars_result}")

# Clean up
duck_conn.close()